## Model architectures and training phase
This section deals with the models creation, the performance results and the dicussion of them.





In [ ]:
!pip install numpy opencv-python matplotlib

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import pandas as pd
import numpy as np
import pathlib

In [ ]:
import tensorflow as tf
from keras import layers, models
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.model_selection import train_test_split

# Data Loading

In [ ]:
dataset_url = "http://download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
data_dir = pathlib.Path(data_dir)

Getting IMages, Labels, and class names from the directory folder

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder

# Define the directory path where the images are stored
#image_dir = '/content/flower_photos/'  # Change this to the actual path

# Load and preprocess the image data
def preprocess_images(folder_path, image_size=(128,128)):
    images = []
    labels = []
    class_names = []

    for class_name in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):
            for image_file in os.listdir(class_folder):
                image_path = os.path.join(class_folder, image_file)
                img = Image.open(image_path)
                img = img.resize(image_size)
                img_array = np.array(img)
                images.append(img_array)
                labels.append(class_name)
                if class_name not in class_names:
                    class_names.append(class_name)

    # Convert the labels to numerical values using LabelEncoder
    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(labels)
    y = np.array(Y)

    # Convert the list of images to a NumPy array
    X = np.array(images)

    return X, y, class_names

# Load and preprocess the images and labels
X, y, class_names = preprocess_images(data_dir)

# Verify the shape of X and Y
print("Shape of X:", X.shape)  # (num_samples, 224, 242, 3) for RGB images
print("Shape of Y:", y.shape)  # (num_samples,) representing the labels as numerical values
print("Class names:", class_names)  # List of class names (folder names)

# Now, you can use X and Y as input data and target data for your model.
# The images have been resized to 224x224, and Y contains the labels represented as numerical values.


#Preprocesisng Data

In [ ]:
x=X/255        # scaling of each image pixel

#Here We are performing Splitting Data into Train , Test and Validation. Data Augmentation

In [ ]:
#pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=0)

# Create a Sequential model for data augmentation
#data_augmentation = keras.Sequential([
#    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=x_train.shape[1:]),
#    layers.experimental.preprocessing.RandomRotation(0.2),
#    layers.experimental.preprocessing.RandomZoom(0.2),
#    layers.experimental.preprocessing.RandomContrast(0.2),
#     layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
#])

data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=x_train.shape[1:]),
    layers.experimental.preprocessing.RandomRotation(factor=0.1),
    layers.experimental.preprocessing.RandomZoom(height_factor=0.1, width_factor=0.1),
])

# Create TensorFlow datasets for training, validation, and testing
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Add a batch dimension to the input data before applying data augmentation
train_dataset = train_dataset.map(lambda x, y: (tf.expand_dims(x, 0), y))
validation_dataset = validation_dataset.map(lambda x, y: (tf.expand_dims(x, 0), y))

# Apply data augmentation to the training dataset
train_dataset = train_dataset.shuffle(buffer_size=len(x_train))
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
train_dataset = train_dataset.batch(batch_size=16)

# Apply data augmentation to the validation dataset (optional)
#validation_dataset = validation_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
#validation_dataset = validation_dataset.batch(batch_size=16)

# Now, you can use the augmented training and validation datasets in your model training


# Display Few Augmeneted Random Images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


for batch in train_dataset.take(1):
    augmented_images = batch[0]  # First element in the batch is the augmented images
    original_labels = batch[1]   # Second element in the batch is the corresponding labels

# Convert the random indices to a TensorFlow tensor
num_images_to_visualize = 5
random_indices = np.random.choice(len(augmented_images), num_images_to_visualize, replace=False)
random_indices_tensor = tf.constant(random_indices)

# Select the images at the random indices for visualization
selected_images = tf.gather(augmented_images, random_indices_tensor)

# Create a grid to visualize the images
fig, axes = plt.subplots(1, num_images_to_visualize, figsize=(15, 3))

# Plot the images
for i, image in enumerate(selected_images):
    image = tf.squeeze(image)  # Remove the batch dimension (squeeze)
    image = (image.numpy() * 255).astype(np.uint8)  # Scale the image back to [0, 255]
    axes[i].imshow(image)
    axes[i].set_title(f"Augmented Image {i+1}")
    axes[i].axis("off")

plt.show()


#Model 1

In [ ]:
num_classes = 5
model01 = keras.Sequential([
      # Include data augmentation as the first layer
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=x_train.shape[1:]),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Dropout layer to reduce overfitting
    layers.Dense(256, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])




model01.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model01.summary()

# Define a generator function for training data

In [ ]:

def data_generator(x_train, y_train, batch_size):
    num_samples = len(x_train)
    indices = np.arange(num_samples)

    while True:
        np.random.shuffle(indices)

        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            batch_x = x_train[batch_indices]
            batch_y = y_train[batch_indices]
            yield batch_x, batch_y

# Define batch size and create the data generator
batch_size = 16
train_generator = data_generator(x_train, y_train, batch_size)

# Fit the model using the data generator and include validation data
steps_per_epoch = len(x_train) // batch_size
validation_dataset = (x_val, y_val)  # Use the validation data you already have

model01.fit(train_generator, epochs=30, steps_per_epoch=steps_per_epoch, validation_data=validation_dataset)


In [ ]:

tf.keras.utils.plot_model(model01,show_shapes=True,show_layer_names=True)

# Plot training and validation Accuracy AND Loss over epochs

In [ ]:
import matplotlib.pyplot as plt


history = model01.history.history

# Plot training and validation loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plot training and validation accuracy over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()


# Assuming you have already trained the model and have x_test and y_test ready

# Evaluate the model on the test dataset

In [ ]:

test_loss, test_accuracy = model01.evaluate(x_test, y_test)

# Print the test loss and accuracy
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


#Make Prediction on train data for Target

In [ ]:
import numpy as np


num_images_to_predict = 5
random_indices = np.random.choice(len(x_test), num_images_to_predict, replace=False)
selected_images = x_test[random_indices]

# Make predictions on the selected images
predictions = model01.predict(selected_images)

# Convert predictions to class labels (if you have a classification problem)
predicted_classes = np.argmax(predictions, axis=1)

# Display the predictions
for i in range(num_images_to_predict):
    print(f"Image {i+1}:")
    print(f"True Label: {y_test[random_indices[i]]}")
    print(f"Predicted Class: {predicted_classes[i]}")
    print("\n")


# Visualizing some randomly chosen indices from the test datasett



In [ ]:
import numpy as np
import matplotlib.pyplot as plt




num_images_to_predict = 5
random_indices = np.random.choice(len(x_test), num_images_to_predict, replace=False)
selected_images = x_test[random_indices]
true_labels = y_test[random_indices]
predicted_classes = np.argmax(predictions, axis=1)

# Create a mapping for class labels (if applicable)

# Display the images with true and predicted labels
plt.figure(figsize=(15, 10))
for i in range(num_images_to_predict):
    plt.subplot(1, num_images_to_predict, i + 1)
    plt.imshow(selected_images[i])
    plt.title(f"True: {class_names[true_labels[i]]}\nPredicted: {class_names[predicted_classes[i]]}")
    plt.axis("off")

plt.show()


# Make predictions on the test dataset using the trained Keras model

In [ ]:
#Make predictions on the test dataset using the trained Keras model
import numpy as np
from sklearn.metrics import classification_report

# Assuming you have already trained the model and have x_test, y_test, and predicted_classes ready

predictions = model1.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# Print the classification report
 # Replace with your class names
print(classification_report(y_test, predicted_classes, target_names=class_names))


# Model2 with regularization, Batch normalization and double Dense layer

In [ ]:
from keras import regularizers
from keras.layers import BatchNormalization

num_classes = 5
model2 = keras.Sequential([
    data_augmentation,  # Include data augmentation as the first layer
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=x_train.shape[1:], padding='same'),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),

    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Dropout(0.5),


#    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
#    layers.BatchNormalization(),
#    layers.Dropout(0.5),

    layers.Dense(num_classes, activation='softmax')
])


model2.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


This is a new possibilitty UP||

In [ ]:
from keras import regularizers
from keras.layers import BatchNormalization

num_classes = 5
model02 = keras.Sequential([
    data_augmentation,  # Include data augmentation as the first layer
    layers.Conv2D(64, (3,3), activation='relu', input_shape=x_train.shape[1:], padding='same'),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),

    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Dropout(0.5),


#    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
#    layers.BatchNormalization(),
#    layers.Dropout(0.5),

    layers.Dense(num_classes, activation='softmax')
])


model02.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model2.summary()



In [ ]:
def data_generator(x_train, y_train, batch_size):
    num_samples = len(x_train)
    indices = np.arange(num_samples)

    while True:
        np.random.shuffle(indices)

        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            batch_x = x_train[batch_indices]
            batch_y = y_train[batch_indices]
            yield batch_x, batch_y

# Define batch size and create the data generator
batch_size = 16
train_generator = data_generator(x_train, y_train, batch_size)

# Fit the model using the data generator and include validation data
steps_per_epoch = len(x_train) // batch_size
validation_dataset = (x_val, y_val)  # Use the validation data you already have

model2.fit(train_generator, epochs=30, steps_per_epoch=steps_per_epoch, validation_data=validation_dataset)


In [ ]:

tf.keras.utils.plot_model(model2,show_shapes=True,show_layer_names=True)


# # Plot training and validation Accuracy AND Loss over epochs

In [ ]:
import matplotlib.pyplot as plt


history = model2.history.history

# Plot training and validation loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plot training and validation accuracy over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Get the training history from the model
history = model02.history.history

# Plot training and validation loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plot training and validation accuracy over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()


From the plot of the accuracy, you can see that the model could probably be trained a little more as the trend for accuracy on both datasets is still rising for the last few epochs. You can also see that the model has not yet over-learned the training dataset, showing comparable skill on both datasets.

#Evaluate the model on the test dataset

In [ ]:

test_loss, test_accuracy = model2.evaluate(x_test, y_test)

# Print the test loss and accuracy
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

#Make Prediction on train data for Target

In [ ]:

import numpy as np


num_images_to_predict = 5
random_indices = np.random.choice(len(x_test), num_images_to_predict, replace=False)
selected_images = x_test[random_indices]

# Make predictions on the selected images
predictions = model2.predict(selected_images)

# Convert predictions to class labels (if you have a classification problem)
predicted_classes = np.argmax(predictions, axis=1)

# Display the predictions
for i in range(num_images_to_predict):
    print(f"Image {i+1}:")
    print(f"True Label: {y_test[random_indices[i]]}")
    print(f"Predicted Class: {predicted_classes[i]}")
    print("\n")

# Visualize some randomly chosen indices from the test dataset

In [ ]:

import numpy as np
import matplotlib.pyplot as plt



num_images_to_predict = 5
random_indices = np.random.choice(len(x_test), num_images_to_predict, replace=False)
selected_images = x_test[random_indices]
true_labels = y_test[random_indices]
predicted_classes = np.argmax(predictions, axis=1)
# Create a mapping for class labels (if applicable)

# Display the images with true and predicted labels
plt.figure(figsize=(15, 10))
for i in range(num_images_to_predict):
    plt.subplot(1, num_images_to_predict, i + 1)
    plt.imshow(selected_images[i])
    plt.title(f"True: {class_names[true_labels[i]]}\nPredicted: {class_names[predicted_classes[i]]}")
    plt.axis("off")

plt.show()


#Make predictions on the test dataset using the trained Keras model

In [ ]:

import numpy as np
from sklearn.metrics import classification_report


predictions = model2.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# Print the classification report
 # Replace with your class names
print(classification_report(y_test, predicted_classes, target_names=class_names))

## **Model3**

In [ ]:
num_classes = 5
model003 = keras.Sequential([
    data_augmentation,
    layers.Conv2D(64, 7, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),

    layers.Conv2D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),

    # Note the explicit import of Conv2D from keras.layers
    keras.layers.Conv2D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),

    layers.MaxPooling2D(3, strides=1, padding="same"),
    layers.Conv2D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),

    layers.Conv2D(64, 3, activation="relu"),
    layers.GlobalMaxPooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),


    layers.Dense(num_classes, activation="softmax")
])

model003.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


TESTing Other possibilities

In [ ]:
from keras import regularizers
from keras.layers import BatchNormalization

num_classes = 5
model3 = keras.Sequential([
    data_augmentation,
    layers.Conv2D(64,  (7,7), padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D(3, strides=1, padding="same"),

    layers.Conv2D(64,  (3,3), padding="same",kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(128,  (3,3), padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling2D(3, strides=1, padding="same"),

    layers.Conv2D(128, (3,3), padding="same",kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(128,  (3,3), padding="same",kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Conv2D(512, (3,3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.GlobalMaxPooling2D(),

#    layers.Conv2D(512, 3, padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#    layers.BatchNormalization(),
#    layers.Activation("relu"),
#    layers.GlobalMaxPooling2D(),

    # ADD FLATTEN LAYER
    layers.Flatten(),
    # ADD DROPOUT LAYER
    layers.Dropout(.5),

    layers.Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001))])



layers.Dense(num_classes, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(0.001))


model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




In [ ]:
model3.summary()

In [ ]:
# Define a generator function for training data
def data_generator(x_train, y_train, batch_size):
    num_samples = len(x_train)
    indices = np.arange(num_samples)

    while True:
        np.random.shuffle(indices)

        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            batch_x = x_train[batch_indices]
            batch_y = y_train[batch_indices]
            yield batch_x, batch_y

# Define batch size and create the data generator
batch_size = 16
train_generator = data_generator(x_train, y_train, batch_size)

# Fit the model using the data generator and include validation data
steps_per_epoch = len(x_train) // batch_size
validation_dataset = (x_val, y_val)  # Use the validation data you already have

model003.fit(train_generator, epochs=30, steps_per_epoch=steps_per_epoch, validation_data=validation_dataset)


# Plot training and validation Accuracy AND Loss over epochs

In [ ]:
import matplotlib.pyplot as plt

# Get the training history from the model
history = model003.history.history

plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plot training and validation accuracy over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()


#Evaluate the model on the test dataset

In [ ]:

test_loss, test_accuracy = model3.evaluate(x_test, y_test)

# Print the test loss and accuracy
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

#Make Prediction on train data for Target

In [ ]:
import numpy as np

# Get some randomly chosen indices from the test dataset
num_images_to_predict = 5
random_indices = np.random.choice(len(x_test), num_images_to_predict, replace=False)
selected_images = x_test[random_indices]

# Make predictions on the selected images
predictions = model3.predict(selected_images)

# Convert predictions to class labels (if you have a classification problem)
predicted_classes = np.argmax(predictions, axis=1)

# Display the predictions
for i in range(num_images_to_predict):
    print(f"Image {i+1}:")
    print(f"True Label: {y_test[random_indices[i]]}")
    print(f"Predicted Class: {predicted_classes[i]}")
    print("\n")

#Make predictions on the test dataset using the trained Keras model

In [ ]:

import numpy as np
from sklearn.metrics import classification_report

# Assuming you have already trained the model and have x_test, y_test, and predicted_classes ready

predictions = model3.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# Print the classification report
 # Replace with your class names
print(classification_report(y_test, predicted_classes, target_names=class_names))

#Model4

In [ ]:
"""
num_classes = 5
model4 = keras.Sequential([
data_augmentation,

layers.Conv2D(32, 3, padding="same", kernel_initializer="he_uniform", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.MaxPooling2D(3, strides=3, padding="same"),

layers.Conv2D(64, 3, padding="same", kernel_initializer="he_uniform", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.MaxPooling2D(3, strides=3, padding="same"),

layers.Conv2D(128, 3, padding="same", kernel_initializer="he_uniform", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.MaxPooling2D(3, strides=3, padding="same"),

layers.Conv2D(128, 3, padding="same", kernel_initializer="he_uniform", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.MaxPooling2D(3, strides=3, padding="same"),

layers.Conv2D(512, 3, padding="same", kernel_initializer="he_uniform", kernel_regularizer=tf.keras.regularizers.l2(0.001)),

layers.BatchNormalization(),
layers.Activation("relu"),
layers.GlobalMaxPooling2D(),

# ADD FLATTEN LAYER
layers.Flatten(),
# ADD DROPOUT LAYER
layers.Dropout(.15),

layers.Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)),
layers.Dropout(.15),


layers.Dense(num_classes, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(0.001))])


# PRINT THE SUMMARY
model4.summary()


model4.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])"""




In [ ]:
from keras import regularizers
from keras.layers import BatchNormalization

num_classes = 5
model_large_cnn = keras.Sequential([
    data_augmentation,  # Include data augmentation as the first layer

    # First convolutional block
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=x_train.shape[1:], padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(3, strides=3, padding="same"),
    layers.Dropout(0.25),

    # Second convolutional block
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(3, strides=3, padding="same"),
    layers.Dropout(0.25),

    # Third convolutional block
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(3, strides=3, padding="same"),
    layers.Dropout(0.25),

    # Fourth convolutional block
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    #layers.MaxPooling2D(3, strides=3, padding="same"),
    layers.GlobalMaxPooling2D(),
    layers.Dropout(0.25),


    # Flatten the output and feed into dense layers
    layers.Flatten(),
    # First fully connected block
    layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.25),
    layers.BatchNormalization(),
    layers.Dropout(0.25),

    # Second fully connected block
    layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.25),
    layers.BatchNormalization(),
    layers.Dropout(0.25),

    # Output layer
    layers.Dense(num_classes, activation='softmax')
])

model_large_cnn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_large_cnn.summary()

In [ ]:
# Define a generator function for training data
def data_generator(x_train, y_train, batch_size):
    num_samples = len(x_train)
    indices = np.arange(num_samples)

    while True:
        np.random.shuffle(indices)

        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            batch_x = x_train[batch_indices]
            batch_y = y_train[batch_indices]
            yield batch_x, batch_y

# Define batch size and create the data generator
batch_size = 16
train_generator = data_generator(x_train, y_train, batch_size)

# Fit the model using the data generator and include validation data
steps_per_epoch = len(x_train) // batch_size
validation_dataset = (x_val, y_val)

model_large_cnn.fit(train_generator, epochs=30, steps_per_epoch=steps_per_epoch, validation_data=validation_dataset)


# Plot training and validation Accuracy AND Loss over epochs

In [ ]:
import matplotlib.pyplot as plt


history = model_large_cnn.history.history

# Plot training and validation loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plot training and validation accuracy over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()


#Evaluate the model on the test dataset

In [ ]:

test_loss, test_accuracy = model_large_cnn.evaluate(x_test, y_test)

# Print the test loss and accuracy
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

#Make Prediction on train data for Target


In [ ]:

import numpy as np


num_images_to_predict = 5
random_indices = np.random.choice(len(x_test), num_images_to_predict, replace=False)
selected_images = x_test[random_indices]

# Make predictions on the selected images
predictions = model_large_cnn.predict(selected_images)

# Convert predictions to class labels (if you have a classification problem)
predicted_classes = np.argmax(predictions, axis=1)

# Display the predictions
for i in range(num_images_to_predict):
    print(f"Image {i+1}:")
    print(f"True Label: {y_test[random_indices[i]]}")
    print(f"Predicted Class: {predicted_classes[i]}")
    print("\n")

#Make predictions on the test dataset using the trained Keras model


In [ ]:
import numpy as np
from sklearn.metrics import classification_report



predictions = model_large_cnn.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# Print the classification report
 # Replace with your class names
print(classification_report(y_test, predicted_classes, target_names=class_names))


## **Transfer Learning VGG16 PRE-TRAINED MODEL**

In [ ]:
##### TRANSFORM LEARNING FROM VGG16 PRE-TRAINED MODEL

In [ ]:
from keras.applications import VGG16


base_model = VGG16(weights='imagenet', include_top=False, input_shape=x_train.shape[1:])


In [ ]:
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
from keras.models import Model


custom_head = model_large_cnn.layers[-7:]

# Extract the output tensor from the base model
base_model_output = model_large_cnn.layers[-8].output  # Assuming the last layer before dropout is the actual output

# Apply the custom head to the base model output
transfer_model = base_model_output
for layer in custom_head:
    transfer_model = layer(transfer_model)

# Create the transfer learning model
transfer_model = Model(inputs=model_large_cnn.input, outputs=transfer_model)


In [ ]:
transfer_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Make sure to have x_train, y_train, x_val, y_val prepared accordingly

# Fit the transfer learning model with your data
transfer_model.fit(train_generator,
                   steps_per_epoch=x_train.shape[0] // batch_size,
                   epochs=30,
                   validation_data=(x_val, y_val))


In [ ]:
import matplotlib.pyplot as plt

history = transfer_model.history.history

# Plot training and validation loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.ylim(0, max(history['loss'] + history['val_loss']))  # Set y-axis limits to start from 0
plt.show()

# Plot training and validation accuracy over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.ylim(0, 1)  # Set y-axis limits for accuracy between 0 and 1
plt.show()


In [ ]:
print(transfer_model.summary())

# `***RESTNET Transfer Learning***`

In [ ]:
from keras.applications import ResNet50

# Load the pre-trained ResNet50 model without the top (fully connected) layers
ResNet50 = ResNet50(weights='imagenet', include_top=False, input_shape=x_train.shape[1:])


In [ ]:
for layer in ResNet50.layers:
    layer.trainable = False


In [ ]:
from keras.models import Model

custom_head = model_large_cnn.layers[-7:]

# Extract the output tensor from the base model
base_model_output = model_large_cnn.layers[-8].output

# Apply the custom head to the base model output
ResNet50_Model = base_model_output
for layer in custom_head:
    ResNet50_Model = layer(ResNet50_Model)

# Create the transfer learning model
ResNet50_Model = Model(inputs=model_large_cnn.input, outputs=ResNet50_Model)



In [ ]:
ResNet50_Model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Fit the transfer learning model with your data
ResNet50_Model.fit(train_generator,
                   steps_per_epoch=x_train.shape[0] // batch_size,
                   epochs=30,
                   validation_data=(x_val, y_val))


In [ ]:
print(ResNet50_Model.summary())

In [ ]:
import matplotlib.pyplot as plt

history = ResNet50_Model.history.history

# Plot training and validation loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.ylim(0, max(history['loss'] + history['val_loss']))  # Set y-axis limits to start from 0
plt.show()

# Plot training and validation accuracy over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.ylim(0, 1)  # Set y-axis limits for accuracy between 0 and 1
plt.show()


# **DenseNet Pre-trained Model**

In [ ]:
from keras.applications import DenseNet121

densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=x_train.shape[1:])


In [ ]:
from keras.models import Model

custom_head = model_large_cnn.layers[-7:]

# Extract the output tensor from the base model
base_model_output = model_large_cnn.layers[-8].output

# Applying the custom head to the base model output
densenet_model = base_model_output
for layer in custom_head:
    densenet_model = layer(densenet_model)

# Create the transfer learning model
densenet_model = Model(inputs=model_large_cnn.input, outputs=densenet_model)

densenet_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:

# Fit the transfer learning model with your data
densenet_model.fit(train_generator,
                   steps_per_epoch=x_train.shape[0] // batch_size,
                   epochs=30,
                   validation_data=(x_val, y_val))


In [ ]:
import matplotlib.pyplot as plt


history = densenet_model.history.history

# Plot training and validation loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.ylim(0, max(history['loss'] + history['val_loss']))  # Set y-axis limits to start from 0
plt.show()

# Plot training and validation accuracy over epochs
plt.figure(figsize=(8, 6))
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.ylim(0, 1)  # Set y-axis limits for accuracy between 0 and 1
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


y_pred = densenet_model.predict(x_val)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_val, y_pred_labels)

classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap='viridis', values_format='d')
plt.title('Confusion Matrix')
plt.show()





In [ ]:
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
len(tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////